In [1]:
import pandas as pd
import datetime as dt
import redis
import json
def get_fills_from_redis(strat = 'capital_txo_main', acc='TAIFEX100',date=dt.date.today(),night_session=False, redishost='prod1.capital.radiant-knight.com'):
    r = redis.StrictRedis(host=redishost, port=6379,db=0)
    rk = '{}:{}'.format(acc, date.strftime('%Y%m%d'))
    rk_strat = '{}:{}'.format(strat, date.strftime('%Y%m%d'))
    if night_session:
        rk = rk + 'E'
        rk_strat = rk_strat + 'E'
    if rk not in [k.decode() for k in r.keys()]:
        print('Key:{} not exists!'.format(rk))
        if rk_strat in [k.decode() for k in r.keys()]:
            rk = rk_strat
        else:
            return None
    msgs = r.lrange(rk, 0, -1)
    trd_df = pd.DataFrame([json.loads(m.decode()) for m in msgs])
    if trd_df.empty:
        print('Empty redis key')
        return None
    if trd_df['ts'][0] > 1e11:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x/1e6))
    else:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x))
    trd_df.set_index('time',inplace=True)

    return trd_df
# ======== 計算手續費函數 =========
def compute_fee(product, px, sz, mult):
    tax_rate = 0.00002
    fee_map = {
        'TXF': 20,
        'MXF': 12.5,
        'TMF': 8.0,
        'FXF': 20,
        'ZFF': 8.0
    }
    fix_fee = fee_map.get(product[:3], 0)
    if fix_fee == 0:
        print('Unknown product:', product)
    tax_fee = abs(px * sz * mult) * tax_rate
    return abs(sz * fix_fee) + tax_fee

# ======== 單一盤別損益計算（日 or 夜） =========
def daily_pnl_neutrino(strategy, d, night_session=False):
    trade_fill = get_fills_from_redis(strategy, date=d, night_session=night_session).reset_index()
    floor = '6f'  # 固定只做六樓

    pnl_records = []
    fill_records = []

    if trade_fill.empty:
        return pd.DataFrame(columns=['floor', 'expiry', 'pnl(twd)']), pd.DataFrame(columns=['time','floor','expiry','pnl(twd)','type'])
    
    trade_fill['expiry'] = trade_fill['instr'].str[-6:]
    trade_fill['contract'] = trade_fill['instr'].str[-10:-7]

    for expiry, group in trade_fill.groupby('expiry'):
        group = group.sort_values(by='ts')
        pos = value = 0
        context = {'alqortake': '', 'contract': '', 'time': None}

        for _, row in group.iterrows():
            contract, px, sz, mult = row['contract'], row['px'], row['sz'], row['mult']
            pos += sz * mult
            value -= px * mult * sz
            value -= compute_fee(contract, px, sz, mult)

            if pos != 0:
                if not context['alqortake']:
                    context.update({'alqortake': row['alqortake'], 'contract': contract})
                if context['time'] is None:
                    context['time'] = row['time']
            else:
                tp = 'fire' if (context['alqortake'] == 'take' and context['alqortake'] == row['alqortake'] and context['contract'] != contract) else 'quoting'
                fill_records.append([context['time'], floor, expiry, value, tp])
                pnl_records.append([floor, expiry, value])
                value = 0
                context = {'alqortake': '', 'contract': '', 'time': None}

    pnl_df = pd.DataFrame(pnl_records, columns=['floor', 'expiry', 'pnl(twd)'])
    pnl_fill = pd.DataFrame(fill_records, columns=['time', 'floor', 'expiry', 'pnl(twd)', 'type'])
    return pnl_df, pnl_fill

# ======== 合併日盤＋夜盤，視為一整天 =========
def daily_pnl_neutrino_full_day(strategy, d):
    all_pnl = pd.DataFrame(columns=['floor', 'expiry', 'pnl(twd)'])
    all_fill = pd.DataFrame(columns=['time','floor','expiry','pnl(twd)','type'])

    for night in [False, True]:
        pnl_df, pnl_fill = daily_pnl_neutrino(strategy, d, night_session=night)
        all_pnl = pd.concat([all_pnl, pnl_df], axis=0)
        all_fill = pd.concat([all_fill, pnl_fill], axis=0)

    return all_pnl, all_fill

# ======== 執行主流程：遍歷整月 6 月 =========
date_range = pd.date_range(start='2025-05-01', end='2025-06-30')
strategy = 'capital_neutrino_txf'  # 只分析六樓

# 初始化統計表
fire_pnl = pd.DataFrame(columns=['date', 'floor', 'pnl(twd)'])
contract_pnl = pd.DataFrame(columns=['time','floor','expiry','pnl(twd)','type'])
alqorfire_count = pd.DataFrame(columns=['time','floor','expiry','pnl(twd)','type'])
full_day_6f = pd.DataFrame(columns=['time','floor','expiry','pnl(twd)','type','date'])

for d in date_range:
    try:
        result1, result2 = daily_pnl_neutrino_full_day(strategy, d.date())
        if result2.empty:
            continue

        f6 = result2[(result2['floor'] == '6f') & (result2['type'] == 'fire')]
        f6_pnl = f6['pnl(twd)'].sum() if not f6.empty else 0

        contract_pnl = pd.concat([contract_pnl, f6], axis=0)
        alqorfire_count = pd.concat([alqorfire_count, result2], axis=0)
        fire_pnl.loc[len(fire_pnl)] = [d, '6f', f6_pnl]

        f6 = f6.copy()
        f6['date'] = d.date()
        full_day_6f = pd.concat([full_day_6f, f6], axis=0)

    except Exception as e:
        print(f"Error processing {d.date()}: {e}")
        continue


Key:TAIFEX100:20250501 not exists!
Error processing 2025-05-01: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250502 not exists!
Key:TAIFEX100:20250502E not exists!
Key:TAIFEX100:20250503 not exists!
Error processing 2025-05-03: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250504 not exists!
Error processing 2025-05-04: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250505 not exists!
Key:TAIFEX100:20250505E not exists!
Key:TAIFEX100:20250506 not exists!


/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pnl = pd.concat([all_pnl, pnl_df], axis=0)
/tmp/ipykernel_73368/411648843.py:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_fill = pd.concat([all_fill, pnl_fill], axis=0)
/tmp/ipykernel_73368/411648843.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the re

Key:TAIFEX100:20250506E not exists!
Key:TAIFEX100:20250507 not exists!
Key:TAIFEX100:20250507E not exists!
Key:TAIFEX100:20250508 not exists!
Key:TAIFEX100:20250508E not exists!
Key:TAIFEX100:20250509 not exists!
Key:TAIFEX100:20250509E not exists!
Key:TAIFEX100:20250510 not exists!


/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pnl = pd.concat([all_pnl, pnl_df], axis=0)
/tmp/ipykernel_73368/411648843.py:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_fill = pd.concat([all_fill, pnl_fill], axis=0)
/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the res

Error processing 2025-05-10: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250511 not exists!
Error processing 2025-05-11: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250512 not exists!
Key:TAIFEX100:20250512E not exists!
Key:TAIFEX100:20250513 not exists!
Key:TAIFEX100:20250513E not exists!
Key:TAIFEX100:20250514 not exists!
Error processing 2025-05-14: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250515 not exists!
Key:TAIFEX100:20250515E not exists!


...

/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pnl = pd.concat([all_pnl, pnl_df], axis=0)
/tmp/ipykernel_73368/411648843.py:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_fill = pd.concat([all_fill, pnl_fill], axis=0)
/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the res

Key:TAIFEX100:20250618E not exists!
Key:TAIFEX100:20250619 not exists!
Key:TAIFEX100:20250619E not exists!
Key:TAIFEX100:20250620 not exists!
Key:TAIFEX100:20250620E not exists!
Key:TAIFEX100:20250621 not exists!
Error processing 2025-06-21: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250622 not exists!
Error processing 2025-06-22: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250623 not exists!
Key:TAIFEX100:20250623E not exists!
Key:TAIFEX100:20250624 not exists!
Key:TAIFEX100:20250624E not exists!
Key:TAIFEX100:20250625 not exists!
Key:TAIFEX100:20250625E not exists!
Key:TAIFEX100:20250626 not exists!


/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pnl = pd.concat([all_pnl, pnl_df], axis=0)
/tmp/ipykernel_73368/411648843.py:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_fill = pd.concat([all_fill, pnl_fill], axis=0)
/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the res

Key:TAIFEX100:20250626E not exists!
Key:TAIFEX100:20250627 not exists!
Key:TAIFEX100:20250627E not exists!
Key:TAIFEX100:20250628 not exists!
Error processing 2025-06-28: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250629 not exists!
Error processing 2025-06-29: 'NoneType' object has no attribute 'reset_index'
Key:TAIFEX100:20250630 not exists!
Key:TAIFEX100:20250630E not exists!


/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pnl = pd.concat([all_pnl, pnl_df], axis=0)
/tmp/ipykernel_73368/411648843.py:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_fill = pd.concat([all_fill, pnl_fill], axis=0)
/tmp/ipykernel_73368/411648843.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the res

In [2]:
if not alqorfire_count.empty:
    df = alqorfire_count.copy()
    df['ex_time'] = pd.to_datetime(df['time'])
    df['date'] = (df['ex_time'] - pd.Timedelta(hours=6)).dt.date  # 以交易邏輯日分組

    df_summary = (
        df
        .groupby(['date', 'floor', 'type'])
        .agg(
            pnl_twd=('pnl(twd)', 'sum'),
            count=('pnl(twd)', 'count')
        )
        .round(0)
        .astype(int)
        .unstack(level=['floor', 'type'])
    )

    df_summary.loc['sum'] = df_summary.sum()
    df_summary.fillna(0, inplace=True)
    df_summary['count'] = df_summary['count'].astype(int)

    with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.float_format", "{:,.0f}".format):
        display(df_summary)
else:
    print("⚠️ No fill data available for the given date range.")


pnl_twd         count        
floor           6f            6f        
type          fire quoting  fire quoting
date                                    
2025-04-30   7,493   9,582    14      27
2025-05-02  -3,000  13,456     3      20
2025-05-05    -615  10,720     4      24
2025-05-06       0   2,078     0      26
2025-05-07  -3,369  11,432     7      26
2025-05-08       0   6,243     0      26
2025-05-09     886   2,109     4      10
2025-05-12  -3,665   6,191     3      19
2025-05-13       0  -1,257     0       5
2025-05-14       0     711     0       2
2025-05-15       0     112     0       9
2025-05-16   2,666     658     2       7
2025-05-19    -544   3,049     1      14
2025-05-20     822   3,375     2      10
2025-05-21   5,522  22,695     7      37
2025-05-22       0     919     0       5
2025-05-23  -1,327  -5,164     3      26
2025-05-26     559   1,163     1      10
2025-05-27     -37     722     6      18
2025-05-28     676   2,870     3       7
2025-05-29     994   1,011     5      10
2025-06-02     343      -4     3      28
2025-06-03  -1,834    -126     1      10
2025-06-04    -772  -1,242     3       9
2025-06-05  -1,616 -13,558     4      29
2025-06-06   1,465  -1,387     2      21
2025-06-09    -329   2,459     3      16
2025-06-10  -1,245   4,381     1       8
2025-06-11       0   3,118     0      13
2025-06-12       0     373     0       7
2025-06-13  -7,221     182     4      13
2025-06-16   1,369   6,239     3      17
2025-06-17       0    -155     0      15
2025-06-18  -1,128   3,638     4      47
2025-06-19       0   5,407     0      21
2025-06-20  -2,171  -5,802     4      73
2025-06-23       0     361     0      63
2025-06-24       0  -3,894     0      31
2025-06-25    -997   1,879     1      10
2025-06-26       0   1,589     0      12
2025-06-27   2,538   2,326     6       9
2025-06-30       0   1,108     0       3
sum         -4,537  99,567   104     793